In [9]:
%%writefile finetune_phi3_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./src

command: >- 
  echo "python finetune_phi3.py ${{inputs.data_dir}} ${{outputs.out_dir}}"; sleep 60000
inputs:
  data_dir: 
    type: uri_folder
    #mode: ro_mount
    #path: azureml:burbery_data@latest
    path: azureml:Sujet-Finance-Vision-10k@latest
outputs:
  out_dir: 
    type: custom_model
    mode: upload
environment: azureml:llava_finetuning:16
environment_variables:
    WANDB_MODE: disabled
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 1
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
#compute: azureml:fine-tune-cluster
compute: azureml:a100-low-priority-france
display_name: finetune_phi3_vision_job_5
#name: finetune_phi3_vision_job
experiment_name: finetune_phi3_vision
description: Finetune Phi3 Vision


Overwriting finetune_phi3_job.yaml


In [10]:
!az ml job create -f finetune_phi3_job.yaml

{
  "code": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServices/workspaces/gpu-workspace/codes/96cc4502-0b66-4371-bcac-37f0da9e105e/versions/1",
  "command": "echo \"python finetune_phi3.py ${{inputs.data_dir}} ${{outputs.out_dir}}\"; sleep 60000",
  "compute": "azureml:a100-low-priority-france",
  "creation_context": {
    "created_at": "2024-06-12T13:46:45.126536+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User"
  },
  "description": "Finetune Phi3 Vision",
  "display_name": "finetune_phi3_vision_job_5",
  "distribution": {
    "process_count_per_instance": 1,
    "type": "pytorch"
  },
  "environment": "azureml:llava_finetuning:16",
  "environment_variables": {
    "WANDB_MODE": "disabled"
  },
  "experiment_name": "finetune_phi3_vision",
  "id": "azureml:/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/antonslutsky-rg/providers/Microsoft.MachineLearningServic

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
%%writefile endpoint_phi3_env_jo.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
command: >- 
  echo "Model Dir: ${{inputs.model_dir}}"; sleep 6000
inputs:
  model_dir: 
    type: custom_model
    path: azureml:finetuned_phi3_vision@latest
environment: azureml:llava_finetuning_inference:2
environment_variables:
    WANDB_MODE: disabled
resources:
  instance_count: 1
distribution:
  type: pytorch 
  process_count_per_instance: 1
services:
    my_vs_code:
      type: vs_code
      nodes: all # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
    my_jupyter_lab:
      type: jupyter_lab
      nodes: all
compute: azureml:fine-tune-cluster
#compute: azureml:a100-low-priority
display_name: endpoint_phi3_env_job
#name: endpoint_phi3_env_job
experiment_name: finetune_phi3_vision
description: Endpoint Phi3 Vision


In [ ]:
!az ml job create -f endpoint_phi3_env_jo.yaml

In [ ]:
%%writefile ./endpoint/endpoint.yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: finetunedPhi3Endpoint
auth_mode: key

In [ ]:
!az ml online-endpoint create --file ./endpoint/endpoint.yaml

In [ ]:
%%writefile ./endpoint/deployment.yaml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: blue
endpoint_name: finetunedPhi3Endpoint
model: azureml:finetuned_phi3_vision:1
code_configuration:
  code: .
  scoring_script: score.py
environment: azureml:llava_finetuning_inference:2
instance_type: Standard_NC12s_v3
instance_count: 1

In [ ]:
!az ml online-deployment create --all-traffic --file ./endpoint/deployment.yaml